In [ ]:
from google.colab import drive

drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Data Science
!ls

/content/drive/.shortcut-targets-by-id/1luUziVf3Yzfx_o_Q34NRaA0dRpJl7pRr/Data Science
 cik_list.xlsx		       'Output Data Structure.xlsx'
 constraining_dictionary.xlsx  'Text Analysis.docx'
 Objective.docx		        uncertainty_dictionary.xlsx


In [ ]:
import re
import nltk
import string
import pandas as pd
import numpy  as np
import urllib.request
from nltk.tokenize import RegexpTokenizer, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
org_file         = pd.read_excel("cik_list.xlsx", header=None) #Reading the data from the Excel file into a DataFrame
header           = org_file.iloc[0,:]                          #The header column names are being fetched 
org_file.columns = header                                      #The header column names are being assigned as the dataframe column names
secfname_or      = org_file.copy()                             #Passing the dataframe values to secfname_or dataframe
secfname_or      = secfname_or.iloc[1:,5]                      #Fetching the SECFNAME column from the dataframe
file_rec         = org_file.iloc[1:,:]                         #1 is given instead of 0 because we do not want header as one of the record
file_rec.head(5)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
1,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,edgar/data/3662/0000950170-98-000413.txt
2,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,edgar/data/3662/0000950170-98-001001.txt
3,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
4,0000003662,SUNBEAM CORP/FL/,199811,1998-11-12 00:00:00,10-K/A,edgar/data/3662/0000950170-98-002145.txt
5,0000003662,SUNBEAM CORP/FL/,199811,1998-11-16 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

#Add the below link at the prefix of all the values in SECFNAME column
link = 'https://www.sec.gov/Archives/'
file_rec['SECFNAME'] = link + file_rec['SECFNAME'].astype(str)

#Now all the values in the SECFNAME column have become hyperlinks linking to 
#HTML pages
file_rec.head(5)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
1,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...
2,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
3,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
4,0000003662,SUNBEAM CORP/FL/,199811,1998-11-12 00:00:00,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...
5,0000003662,SUNBEAM CORP/FL/,199811,1998-11-16 00:00:00,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...


In [ ]:
new_columns = ['mda_positive_score', 'mda_negative_score', 'mda_polarity_score', 'mda_average_sentence_length', 'mda_percentage_of_complex_words', 'mda_fog_index',
               'mda_complex_word_count', 'mda_word_count', 'mda_uncertainty_score', 'mda_constraining_score', 'mda_positive_word_proportion', 'mda_negative_word_proportion',
               'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion','qqdmr_positive_score','qqdmr_negative_score',
               'qqdmr_polarity_score','qqdmr_average_sentence_length','qqdmr_percentage_of_complex_words', 'qqdmr_fog_index','qqdmr_complex_word_count',
               'qqdmr_word_count', 'qqdmr_uncertainty_score', 'qqdmr_constraining_score', 'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion',
               'qqdmr_uncertainty_word_proportion', 'qqdmr_constraining_word_proportion', 'rf_positive_score', 'rf_negative_score',
               'rf_polarity_score', 'rf_average_sentence_length', 'rf_percentage_of_complex_words', 'rf_fog_index', 'rf_complex_word_count',
               'rf_word_count', 'rf_uncertainty_score', 'rf_constraining_score', 'rf_positive_word_proportion', 'rf_negative_word_proportion',
               'rf_uncertainty_word_proportion', 'rf_constraining_word_proportion', 'constraining_words_whole_report']

file_rec = pd.concat([file_rec,pd.DataFrame(columns=new_columns)]) #Appends the above columns to the existing dataframe file_rec
file_rec = file_rec.reset_index()                                  #Resets the row index of the dataframe starting from 0
del file_rec['index']                                              #Deleting the redundant index column from the dataframe

In [ ]:
file_rec.head(3)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Fetching the Stop words list
with open('/content/StopWords_GenericLong.txt') as f:
   stop_words = list(f) 
   stop_words = [s.rstrip() for s in stop_words]
   stop_words = [x.lower() for x in stop_words]

#Fetching the list of Positive and Negative words
positive = pd.read_excel("/content/LoughranMcDonald_SentimentWordLists_2018.xlsx", "Positive", header=None)
negative = pd.read_excel("/content/LoughranMcDonald_SentimentWordLists_2018.xlsx", "Negative", header=None)

#Fetching all the positive and Negative words into two separate lists
positive = positive[0]
negative = negative[0]

#Keeping all the positive and Negative words which are not present in stop words list
positive_cleaned = list(set(positive)-set(stop_words)) 
negative_cleaned = list(set(negative)-set(stop_words)) 

#Converting the Positive and Negative words to Lowercase
positive_cleaned = [x.lower() for x in positive_cleaned]
negative_cleaned = [x.lower() for x in negative_cleaned]

#Fetching the uncertainity and constarining words from the Excel sheet
uncertainity_dict = pd.read_excel("uncertainty_dictionary.xlsx", header=None)
constraining_dict     = pd.read_excel("constraining_dictionary.xlsx", header=None)

#Passing all the above obtained words into separate lists and converting it to Lowercase
uncertainity_words = uncertainity_dict[0].tolist()
uncertainity_words = [x.lower() for x in uncertainity_words]

constraining_words = constraining_dict[0].tolist()
constraining_words = [x.lower() for x in constraining_words]

In [ ]:
###################################################################################################################
#                                FUNCTION FOR FETCHING THE SECTION's DATA                                          |
###################################################################################################################

def fetch_sections_data(data):

################################################################################
#       Fetching the "MANAGEMENT'S DISCUSSION AND ANALYSIS" section             |
################################################################################

  topic1 = "MANAGEMENT'S DISCUSSION AND ANALYSIS"
  topic1_start = [m.start() for m in re.finditer(topic1, data)]

  if (len(topic1_start) == 0):
    topic1 = "MANAGEMENTS DISCUSSION AND ANALYSIS"
    topic1_start = [m.start() for m in re.finditer(topic1, data)]

      
  if (len(topic1_start) != 0):
    #topic_1_data has the data from the point where topic1 begins
    topic_1_data = data[topic1_start[0]:]
    
    item = "ITEM"
    topic1_end = [m.start() for m in re.finditer(item, topic_1_data)]

    if (len(topic1_end) == 0):
      topic_1_data = topic_1_data[0:]
    else:  
      topic_1_data = topic_1_data[0:topic1_end[0]-1]
    #topic_1_data ends at the point where topic1 ends
  if (len(topic1_start) == 0):
    topic_1_data = 0  

#####################################################################################################
#       Fetching the "QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK" section            |
#####################################################################################################

  topic2 = "QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK"
  topic2_start = [m.start() for m in re.finditer(topic2, data)]
  
  if (len(topic2_start) != 0):

    topic_2_data = data[topic2_start[0]:]

    item = "ITEM"
    topic2_end = [m.start() for m in re.finditer(item, topic_2_data)]
    if (len(topic2_end) == 0):
      topic_2_data = topic_2_data[0:]
    else:  
      topic_2_data = topic_2_data[0:topic2_end[0]-1]     
  
  if (len(topic2_start) == 0):
    topic_2_data = 0 

  #SECTION 3 - Risk Factors
  topic3 = "RISK FACTORS"
  topic3_start = [m.start() for m in re.finditer(topic3, data)]

  if (len(topic3_start) != 0):

    topic_3_data = data[topic3_start[0]:]
    
    item = "ITEM"
    topic3_end = [m.start() for m in re.finditer(item, topic_3_data)]
    
    if (len(topic3_end) == 0):
      topic_3_data = topic_3_data[0:]
    else:  
      topic_3_data = topic_3_data[0:topic3_end[0]-1]    

  if (len(topic3_start) == 0):
    topic_3_data = 0         

  return topic_1_data, topic_2_data, topic_3_data

In [ ]:
###################################################################################################################
#                      FUNCTION FOR CALCULATING CONSTRAINING WORDS FOR THE WHOLE REPORT                            |
###################################################################################################################

def constr_whole_report(x):

  x      = x.translate(str.maketrans('','',string.punctuation)) #Removing the Punctuations from the string before tokenizing
  tokens = nltk.word_tokenize(x)                                #Tokenizing the string into a list of words
  tokens = [x.lower() for x in tokens]                          #Converting all the words in thee list to lowercase

  constr_whole_words = 0
  for word in tokens:                                         

    if word in constraining_words:                              #If word is present in constraining word list we increment constr_words by 1                
      constr_whole_words += 1     
      
  return constr_whole_words


In [ ]:

###################################################################################################################
#                  DATA CLEANING - REMOVING THE WORDS PRESENT IN STOP WORDS LIST                                   |
###################################################################################################################

def data_cleaning(x1, x2, x3):

  if (x1 != 0):
    x1 = nltk.word_tokenize(x1)                                     
    x1 = [word for word in x1 if word.lower() not in stop_words]  #Gets all the words which are not present in the stop words list
    x1 = ' '.join(x1)                                             #Combines all the cleaned data into a string of words                                        

  if (x2 != 0):
    x2 = nltk.word_tokenize(x2)                                   #Tokenizes words in the list
    x2 = [word for word in x2 if word.lower() not in stop_words]  #Gets all the words which are not present in the stop words list
    x2 = ' '.join(x2)                                             #Combines all the cleaned data into a string of words                          


  if (x3 != 0):
    x3 = nltk.word_tokenize(x3)                                   #Tokenizes words in the list
    x3 = [word for word in x3 if word.lower() not in stop_words]  #Gets all the words which are not present in the stop words list
    x3 = ' '.join(x3)                                             #Combines all the cleaned data into a string of words

  return x1, x2, x3

In [ ]:
###################################################################################################################
#                 FUNCTION FOR CALCULATING POSITIVE SCORE, NEGATIVE SCORE AND POLARITY SCORE                       |
###################################################################################################################

def pos_neg_pol(x):

  x      = x.translate(str.maketrans('','',string.punctuation)) #Removing the Punctuations from the string before tokenizing 
  tokens = nltk.word_tokenize(x)                                #Tokenizing the string into a list of words
  tokens = [x.lower() for x in tokens]                          #Converting all the words in the list to lower case

  pos_words = neg_words = 0

  for word in tokens:
    if word in positive_cleaned:
      pos_words += 1
    if word in negative_cleaned:
      neg_words -= 1 

  pos_len = pos_words
  neg_len = neg_words * -1

  pol_score = (pos_len - neg_len)/((pos_len + neg_len) + 0.000001)

  return pos_len, neg_len, pol_score

In [ ]:

###################################################################################################################
#                          FUNCTION FOR CALCULATING AVERAGE SENTENCE LENGTH IN SECTION'S DATA                      |
###################################################################################################################

def avg_sent_len(x):

  no_sent = len(sent_tokenize(x))                               #Calculating the number of sentences in the whole data
  x      = x.translate(str.maketrans('','',string.punctuation)) #Removing the Punctuations from the string before tokenizing
  tokens = nltk.word_tokenize(x)                                #Tokenizing the string into a list of words
  no_words = len(tokens)                                        #Calculating the number of words in the ddata

  if (no_sent != 0):
    avg_sent_len = round(no_words/no_sent)                      #Calculating the Average Sentence length
  else:
    avg_sent_len = 0

  return avg_sent_len

In [ ]:

###################################################################################################################
#                    FUNCTION FOR CALCULATING WORD_COUNT, COMPLEX WORD COUNT AND PERC OF COMPLEX WORDS             |
###################################################################################################################

def complex_word_count(word, complex_count):

    word   = word.translate(str.maketrans('','',string.punctuation)) #Removing the Punctuations from the string before tokenizing
    tokens = nltk.word_tokenize(word)                                #Tokenizing using nltk.word_tokenize
    no_words = len(tokens)                                           #Finding out the number of words
    
    complex_count = 0                                                #Initializing the complex_count variable to 0
    for word in tokens:

      word = word.lower()                                            #Converting all the words in the list to Lowercase

      vowels = "aeiou"    

      if (word.endswith(("es", "ed"))):                              #We are ignoring the words ending with es or ed in our calculation
        count = 0

      else:
        count = 0
        
        for c in word:
          if (c in vowels):
            count = count + 1                                        #Counting the number of Vowels in each word

        if (count > 2):                                              #If number of vowels > 2 then we are incrementing complex_count variable by 1
          complex_count = complex_count + 1 

    perc_complex_words = complex_count/no_words                      #Calculating the percentage of complex words

    return no_words, complex_count, perc_complex_words

In [ ]:

###################################################################################################################
#                        FUNCTION FOR CALCULATING UNCERTAINITY AND CONSTRAINING SCORE                              |
###################################################################################################################

def uncer_constr_calc(x):
  
  x      = x.translate(str.maketrans('','',string.punctuation)) #Removing the Punctuations from the string before tokenizing
  tokens = nltk.word_tokenize(x)                                #Tokenizing the string into a list of words
  tokens = [x.lower() for x in tokens]                          #Converting all the words in thee list to lowercase

  uncer_words = constr_words = 0

  for word in tokens:                                         
    if word in uncertainity_words:                              #If word is present in uncertainity word list we increment uncer_words by 1
      uncer_words  += 1
    if word in constraining_words:                              #If word is present in constraining word list we increment constr_words by 1                
      constr_words += 1 

  uncer_count  = uncer_words
  constr_count = constr_words

  return uncer_count, constr_count

In [ ]:
###################################################################################################################
#                                      MAIN PROGRAM OPERATIONS                                                     |
###################################################################################################################

for i in range(len(file_rec)):                    #Looping through the file_rec dataframe
  
  url = file_rec.iloc[i,5]   
  try:                     #Fetching the URL from the dataframe
    data = urllib.request.urlopen(url).readlines()  #Reading the contents from the URL link page
  except:
    continue
  
  #Data cleaning to optimize the speed and efficiency of operation
  data = [str(s, 'utf-8') for s in data]          #Decoding the variable to remove b(bytes object) at the beginning
  data = [s.strip('\n') for s in data]            #Removing the new line character \n at the end of every line
  data = list(filter(None, data))                 #Removing the lines which are blank
  #After doing the above operations the number of iterations required now would be less

  #Converting the list of strings into a single string to increase speed of operation instead of looping again
  data = " ".join(data)

  #Removing the Tables from the data
  data = re.sub("(?is)<table[^>]*>(.*?)<\/table>", "", data)

  #REGEX operations to remove the HTML content from our data and get a clean text data
  html_regex = re.compile(r'<.*?>')
  data = re.sub(html_regex,'',data)
  data = data.replace('&nbsp;','')
  input_data = re.sub(r'&#\d+;', '', data)   

  #Calculating the Constraining words for the whole report
  constr_whole_count = constr_whole_report(input_data)
  file_rec.loc[i, 'constraining_words_whole_report'] = constr_whole_count

  #Fetching the sections data using the fetch_sections_data()
  topic1_data, topic2_data, topic3_data = fetch_sections_data(input_data)

  #Removing the Stop words, punctuations in our data
  topic1_data, topic2_data, topic3_data = data_cleaning(topic1_data, topic2_data, topic3_data)

################################################################################
#   Calculating the values for "MANAGEMENT'S DISCUSSION AND ANALYSIS" section   |
################################################################################

  if (topic1_data != 0):
    
    #Calculating the positive score, negative score and Polarity score
    pos_score1, neg_score1, pol_score1 = pos_neg_pol(topic1_data)

    #Passing the above values to our corresponding dataframe records
    file_rec.loc[i, 'mda_positive_score'] = pos_score1
    file_rec.loc[i, 'mda_negative_score'] = neg_score1 
    file_rec.loc[i, 'mda_polarity_score'] = pol_score1

    #Calculating the average sentence length for our section of data
    avg_sent_len_val1 = avg_sent_len(topic1_data)
    file_rec.loc[i, 'mda_average_sentence_length'] = avg_sent_len_val1

    #Calculating the word count, complex word count and percentage of complex words
    complex_count = 0
    word_count1, complex_count1, perc_complex_words1 = complex_word_count(topic1_data, complex_count)
    file_rec.loc[i, 'mda_word_count']                  = word_count1
    file_rec.loc[i, 'mda_complex_word_count']          = complex_count1
    file_rec.loc[i, 'mda_percentage_of_complex_words'] = perc_complex_words1

    #Calculating the Fog index value 
    fog_index1 = 0.4 * (avg_sent_len_val1 + perc_complex_words1)
    file_rec.loc[i, 'mda_fog_index'] = fog_index1

    #Calculating the uncertainity score and constraining score
    uncer_score1, constr_score1 = uncer_constr_calc(topic1_data)
    perc_uncer1  = uncer_score1/word_count1
    perc_constr1 = constr_score1/word_count1
    file_rec.loc[i, 'mda_uncertainty_score']            = uncer_score1
    file_rec.loc[i, 'mda_constraining_score']           = constr_score1

    #Calculating the Uncertainity and Constraining word proportion
    file_rec.loc[i, 'mda_uncertainty_word_proportion']  = perc_uncer1
    file_rec.loc[i, 'mda_constraining_word_proportion'] = perc_constr1
    
    #Calculating the Positive and Negative word proportion
    perc_positive1 = pos_score1/word_count1
    perc_negative1 = neg_score1/word_count1
    file_rec.loc[i, 'mda_positive_word_proportion']  = perc_positive1
    file_rec.loc[i, 'mda_negative_word_proportion']  = perc_negative1

  else:
    #If the section 1 data is not available, all the below values should be 0
    file_rec.loc[i, 6:21] = 0

#####################################################################################################
#  Calculating the values for "QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK" section   |
#####################################################################################################

  if (topic2_data != 0):  
    
    #Calculating the positive score, negative score and Polarity score
    pos_score2, neg_score2, pol_score2 = pos_neg_pol(topic2_data)
    #Passing the above values to our corresponding dataframe records
    file_rec.loc[i, 'qqdmr_positive_score'] = pos_score2
    file_rec.loc[i, 'qqdmr_negative_score'] = neg_score2 
    file_rec.loc[i, 'qqdmr_polarity_score'] = pol_score2

    #Calculating the average sentence length for our section of data
    avg_sent_len_val2 = avg_sent_len(topic2_data)
    file_rec.loc[i, 'qqdmr_average_sentence_length'] = avg_sent_len_val2

    #Calculating the word count, complex word count and percentage of complex words
    complex_count = 0
    word_count2, complex_count2, perc_complex_words2 = complex_word_count(topic2_data, complex_count)
    file_rec.loc[i, 'qqdmr_word_count']                  = word_count2
    file_rec.loc[i, 'qqdmr_complex_word_count']          = complex_count2
    file_rec.loc[i, 'qqdmr_percentage_of_complex_words'] = perc_complex_words2

    #Calculating the Fog index value 
    fog_index2 = 0.4 * (avg_sent_len_val2 + perc_complex_words2)
    file_rec.loc[i, 'qqdmr_fog_index'] = fog_index2

    #Calculating the uncertainity score and constraining score
    uncer_score2, constr_score2 = uncer_constr_calc(topic2_data)
    perc_uncer2  = uncer_score2/word_count2
    perc_constr2 = constr_score2/word_count2
    file_rec.loc[i, 'qqdmr_uncertainty_score']            = uncer_score2
    file_rec.loc[i, 'qqdmr_constraining_score']           = constr_score2

    #Calculating the Uncertainity and Constraining word proportion
    file_rec.loc[i, 'qqdmr_uncertainty_word_proportion']  = perc_uncer2
    file_rec.loc[i, 'qqdmr_constraining_word_proportion'] = perc_constr2
    
    #Calculating the Positive and Negative word proportion
    perc_positive2 = pos_score2/word_count2
    perc_negative2 = neg_score2/word_count2
    file_rec.loc[i, 'qqdmr_positive_word_proportion']  = perc_positive2
    file_rec.loc[i, 'qqdmr_negative_word_proportion']  = perc_negative2

  else:
    #If the section 2 data is not available, all the below values should be 0
    file_rec.loc[i, 20:33] = 0

################################################################################
#            Calculating the values for "RISK FACTORS" section                  |
################################################################################

  if (topic3_data != 0):  
    
    #Calculating the positive score, negative score and Polarity score
    pos_score3, neg_score3, pol_score3 = pos_neg_pol(topic3_data)
    #Passing the above values to our corresponding dataframe records
    file_rec.loc[i, 'rf_positive_score'] = pos_score3
    file_rec.loc[i, 'rf_negative_score'] = neg_score3 
    file_rec.loc[i, 'rf_polarity_score'] = pol_score3

    #Calculating the average sentence length for our section of data
    avg_sent_len_val3 = avg_sent_len(topic3_data)
    file_rec.loc[i, 'rf_average_sentence_length'] = avg_sent_len_val3

    #Calculating the word count, complex word count and percentage of complex words
    complex_count = 0
    word_count3, complex_count3, perc_complex_words3 = complex_word_count(topic3_data, complex_count)
    file_rec.loc[i, 'rf_word_count']                  = word_count3
    file_rec.loc[i, 'rf_complex_word_count']          = complex_count3
    file_rec.loc[i, 'rf_percentage_of_complex_words'] = perc_complex_words3

    #Calculating the Fog index value 
    fog_index3 = 0.4 * (avg_sent_len_val3 + perc_complex_words3)
    file_rec.loc[i, 'rf_fog_index'] = fog_index3

    #Calculating the uncertainity score and constraining score
    uncer_score3, constr_score3 = uncer_constr_calc(topic3_data)
    perc_uncer3  = uncer_score3/word_count3
    perc_constr3 = constr_score3/word_count3
    file_rec.loc[i, 'rf_uncertainty_score']            = uncer_score3
    file_rec.loc[i, 'rf_constraining_score']           = constr_score3
    #Calculating the Uncertainity and Constraining word proportion
    file_rec.loc[i, 'rf_uncertainty_word_proportion']  = perc_uncer3
    file_rec.loc[i, 'rf_constraining_word_proportion'] = perc_constr3
    
    #Calculating the Positive and Negative word proportion
    perc_positive3 = pos_score3/word_count3
    perc_negative3 = neg_score3/word_count3
    file_rec.loc[i, 'rf_positive_word_proportion']  = perc_positive3
    file_rec.loc[i, 'rf_negative_word_proportion']  = perc_negative3

  else:
    #If the section 1 data is not available, all the below values should be 0
    file_rec.loc[i, 34:47] = 0

  #Clearing all the variables
  pos_score1 = neg_score1 = pol_score1 = avg_sent_len_val1 = word_count1 = complex_count1 = perc_complex_words1 = fog_index1 = uncer_score1 = constr_score1 = perc_uncer1 = perc_constr1 = perc_positive1 = perc_negative1 = 0
  pos_score2 = neg_score2 = pol_score2 = avg_sent_len_val2 = word_count2 = complex_count2 = perc_complex_words2 = fog_index2 = uncer_score2 = constr_score2 = perc_uncer2 = perc_constr2 = perc_positive2 = perc_negative2 = 0
  pos_score3 = neg_score3 = pol_score3 = avg_sent_len_val3 = word_count3 = complex_count3 = perc_complex_words3 = fog_index3 = uncer_score3 = constr_score3 = perc_uncer3 = perc_constr3 = perc_positive3 = perc_negative3 = 0
  constr_whole_count = 0

print("All operations on the dataframe have finished successfully")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:186: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.


All operations on the dataframe have finished successfully


In [ ]:
#Removing the pre-fix http link at the beginning to keep values same as original file
link = 'https://www.sec.gov/Archives/'
secfname_or = secfname_or.reset_index(drop=True)
file_rec['SECFNAME'] = secfname_or 

#Final output is stored in file_rec dataframe
file_rec

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,0000003662,SUNBEAM CORP/FL/,199803,1998-03-06 00:00:00,10-K405,edgar/data/3662/0000950170-98-000413.txt,20,71,-0.56044,21,0.379893,8.55196,854,2248,30,10,0.0088968,0.0315836,0.0133452,0.0044484,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1452
1,0000003662,SUNBEAM CORP/FL/,199805,1998-05-15 00:00:00,10-Q,edgar/data/3662/0000950170-98-001001.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000003662,SUNBEAM CORP/FL/,199808,1998-08-13 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,5
3,0000003662,SUNBEAM CORP/FL/,199811,1998-11-12 00:00:00,10-K/A,edgar/data/3662/0000950170-98-002145.txt,40,131,-0.532164,19,0.412589,7.76504,1678,4067,75,46,0.00983526,0.0322105,0.0184411,0.0113105,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,691
4,0000003662,SUNBEAM CORP/FL/,199811,1998-11-16 00:00:00,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0000012239,SPHERIX INC,200704,2007-04-02 00:00:00,10-K,edgar/data/12239/0001104659-07-024804.txt,109,124,-0.0643777,13,0.388878,5.35555,3867,9944,89,84,0.0109614,0.0124698,0.00895012,0.0084473,85,87,-0.0116279,13,0.38697,5.35479,2946,7613,50,59,0.0111651,0.0114278,0.00656771,0.0077499,134,185,-0.159875,13,0.393503,5.3574,4676,11883,117,107,0.0112766,0.0155685,0.009846,0.00900446,130
148,0000012239,SPHERIX INC,200705,2007-05-16 00:00:00,NT 10-Q,edgar/data/12239/0001104659-07-040463.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0
149,0000012239,SPHERIX INC,200705,2007-05-18 00:00:00,10-Q,edgar/data/12239/0001104659-07-041441.txt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,21
150,0000012239,SPHERIX INC,200705,2007-05-23 00:00:00,10-K/A,edgar/data/12239/0001104659-07-042333.txt,109,124,-0.0643777,13,0.389317,5.35573,3841,9866,89,83,0.011048,0.0125684,0.00902088,0.00841273,85,87,-0.0116279,13,0.388056,5.35522,2924,7535,50,59,0.0112807,0.0115461,0.0066357,0.00783013,134,185,-0.159875,13,0.393868,5.35755,4650,11806,117,106,0.0113502,0.01567,0.00991022,0.00897849,129


In [ ]:
file_rec.to_csv('/content/Final.csv')